In [6]:
import pandas as pd
import re

# CSV 파일 읽기
file_path = "기상/대한민국_기초자치단체_중심점_2017.csv"
df = pd.read_csv(file_path, encoding='cp949')

# 시/군 단위만 추출하는 함수 (예: '용인시기흥구' → '용인시')
def extract_city(sig_name):
    match = re.match(r'^(.*?[시군])', sig_name)
    return match.group(1) if match else sig_name

# 새로운 컬럼에 시/군 이름 저장
df['시군'] = df['SIG_KOR_NM'].apply(extract_city)

# 시군 기준으로 위도/경도 평균 구하고 이름도 해당 시군 이름으로 설정
avg_coords = df.groupby('시군')[['X', 'Y']].mean().reset_index()

# 결과를 SIG_KOR_NM 형식으로 맞추기 위해 컬럼 이름 변경
avg_coords = avg_coords.rename(columns={'시군': 'SIG_KOR_NM'})

# 최종 결과 확인 또는 저장
print(avg_coords)
avg_coords.to_csv("기상/시군_단위_평균_좌표.csv", index=False, encoding='cp949')

    SIG_KOR_NM           X          Y
0          가평군  127.450192  37.818433
1          강남구  127.062985  37.496644
2          강동구  127.147012  37.550450
3          강릉시  128.832379  37.709102
4          강북구  127.011189  37.643474
..         ...         ...        ...
199        홍천군  128.074261  37.745049
200        화성시  126.874858  37.165237
201        화순군  127.033434  35.008186
202        화천군  127.685166  38.138426
203        횡성군  128.077067  37.509142

[204 rows x 3 columns]


In [7]:
import pandas as pd

# 파일 경로
metro_path = "기상/대한민국_광역자치단체_중심점_2017.csv"
siggun_path = "기상/시군_단위_평균_좌표.csv"

# 파일 읽기 (한글 인코딩)
df_metro = pd.read_csv(metro_path, encoding='cp949')
df_siggun = pd.read_csv(siggun_path, encoding='cp949')

# 광역자치단체: 컬럼명 변경 및 재정렬
df_metro = df_metro.rename(columns={"CTP_KOR_NM": "지역"})
df_metro = df_metro[["지역", "X", "Y"]]

# 기초자치단체: 컬럼명 변경
df_siggun = df_siggun.rename(columns={"SIG_KOR_NM": "지역"})

# 두 데이터프레임 병합
df_combined = pd.concat([df_siggun, df_metro], ignore_index=True)

# 최종 컬럼명 변경
df_combined = df_combined.rename(columns={"X": "경도", "Y": "위도"})

# 결과 확인
print(df_combined.head())

# CSV로 저장 (선택)
df_combined.to_csv("기상/전국_자치단체_중심좌표.csv", index=False, encoding='cp949')

    지역          경도         위도
0  가평군  127.450192  37.818433
1  강남구  127.062985  37.496644
2  강동구  127.147012  37.550450
3  강릉시  128.832379  37.709102
4  강북구  127.011189  37.643474
